In [ ]:
%pip install opendatasets
%pip install pandas
import opendatasets as od
import pandas as pd

TODO:
- film ranking
- filter actors by number of films they are in
- separate df for actors

In [ ]:
od.download("https://www.kaggle.com/saikrishnadupp/top3000-imdb-indian-movies")
bollywood = pd.read_csv('top3000-imdb-indian-movies/Top3000_imdb_indian_movies.csv')

In [ ]:
bollywood

DATA PREPROCESSING

In [ ]:
bollywood.dropna(subset=['Crew_dir'], inplace=True)
bollywood.drop(columns=['Date_of_Release'], inplace=True)
bollywood.info()

In [ ]:
bollywood.groupby(['Title']).size().reset_index(name='counts').query('counts > 1') #Probably remakes

In [ ]:
bollywood.describe()

In [ ]:
#JA DZIAŁAM TUTEJ
def find_anomalies(col, dataset, sgm_num = 3):
    col_mean = dataset[col].mean()
    col_std = dataset[col].std()
    col_min = col_mean - sgm_num * col_std
    col_max = col_mean + sgm_num * col_std
    tmp_df = dataset.copy(deep=True)
    tmp_df['col_min'] = col_min
    tmp_df['col_max'] = col_max
    tmp_df = tmp_df.query('' + col + ' > col_max | ' + col + ' < col_min')
    tmp_df = tmp_df.drop(columns=['col_min', 'col_max'])
    return tmp_df

anomalies_votes = find_anomalies('Votes', bollywood)
# anomalies_votes
anomalies_rating = find_anomalies('ImDBRating', bollywood, 2.4)
anomalies_rating.describe()



In [ ]:
# A JA TUTEJ
def split_actors(df_o):
    df = df_o.copy(deep=True)
    actors =  df['Cast_stars']
    df['Cast_stars'] = actors.map(lambda x: x.split(', '))
    df = df.explode('Cast_stars')
    return df
bollywood_actors = split_actors(bollywood)
def calc_weighted_rating(row, mean_rate, min_votes):
    mv = row['Votes'] + min_votes
    return (row['Votes'] / mv) * row['ImDBRating'] + (min_votes / mv) * mean_rate
votes_min = 10000
bollywood_actors_top = bollywood_actors.query('Votes > ' + str(votes_min)).copy()
bollywood_actors_top['weighted_rating'] = bollywood_actors_top.apply(lambda row: calc_weighted_rating(row, bollywood_actors_top['ImDBRating'].mean(), votes_min), axis=1)
bollywood_actors_top.groupby(['Cast_stars']).mean().drop(columns=['Age_of_content']).sort_values(by=['weighted_rating'], ascending=False)